In [27]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import optimize
import matplotlib.pyplot as plt
#import warnings

In [2]:
from Matrix import *
from TLD import *
from AuxFunctions import *

In [3]:
import os

In [4]:
dist_cont = [d for d in dir(stats) if
             isinstance(getattr(stats,d), stats.rv_continuous) and
             hasattr(getattr(stats,d), 'fit')]

In [5]:
#log of distributions not working properly
dist_cont_not_working = ['levy_stable']

In [7]:
dist_cont_working = list(set(dist_cont) - set(dist_cont_not_working))
#dist_continu_working

In [26]:
#TODO: CHECK is Tanner area == !?
def Tanner_func(x, alpha, beta):
    return (x ** beta) * np.exp(alpha * x)

In [18]:
#TEMPORARY: fit method does not work as expected.
class Tanner(stats.rv_continuous):
    def _pdf(self, x, alpha, beta):
        return (x ** beta) * np.exp(alpha * x)
    def _stats(self, alpha, beta):
        #starting values for fit
        return 0., 0., 0., 0.

In [19]:
tanner = Tanner(name='tanner', shapes='alpha, beta')

In [20]:
tanner.fit(Ynorm)

(0.00064354474335964512,
 4.1048820704712746e-08,
 -0.00019229306301766426,
 0.75015919468560166)

In [21]:
class Tanner(stats.rv_continuous):
    def _pdf(self, x, alpha, beta):
        return (x ** beta) * np.exp(alpha * x)
    def _stats(self, alpha, beta):
        #starting values for fit
        return 0., 0., 0., 0.
    #TODO: Review. It would be ideal if Tanner could use built in fit method.
    def fit(self, xdata, ydata, *args, **kwargs):
        return optimize.curve_fit(self._pdf, xdata, ydata, *args, **kwargs)[0]

In [22]:
tanner = Tanner(name='tanner', shapes='alpha, beta')

In [23]:
tanner.fit(X, Ynorm, p0=[-1,5])

array([-3.39581068,  6.73003917])

In [24]:
optimize.curve_fit(Tanner_func, X, Ynorm, p0=[-1,5])

(array([-3.39581068,  6.73003917]), array([[ 0.16631135, -0.47652829],
        [-0.47652829,  1.38310325]]))

In [49]:
#src: http://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python
def fit_distribs(TLD, distrib_names, *args, **kwargs):
    '''Returns a dictionary of distributions fitted for each column in TLD'''
    distribs = {}
    for col in TLD:
        
        distribs[col] = [getattr(stats, fn)(
                            *getattr(stats, fn).fit(TLD[col].dropna(), *args, **kwargs))
                         for fn in distrib_names]
    return distribs

In [62]:
def fit_distribs_to_df(distrib_dict, xdata):
    '''Returns a dataframe with all the distributions in distrib_dict applied to xdata.'''
    df = pd.DataFrame.from_dict(
    {(k, d.dist.name): d.pdf(X) for k,dlst in TLDdistribs.items() for d in dlst})
    df.index = xdata
    return df

In [50]:
TLDdistribs = fit_distribs(normalize_TLD(TLD), tfuncs)

In [65]:
TLDdistribs_df = fit_distribs_to_df(TLDdistribs, X)

In [72]:
for col, dn in TLDdistribs_df:
    OutputName = os.path.join('example_outputs', 'TLD_fit_{}.png'.format(col))
    TLD_to_JPG(TLDdistribs_df.loc[:,col], OutputName)

In [12]:
#TODO
def n_best_fitting_distrib(data, distrib_names, n, *args, **kwargs):
    '''returns the best n fitting distributions of the specified
    list of distirbutions.'''
    ...

In [13]:
def lognorm_mu_sigma(params):
    '''Returns the lognorm parameters (mu, sigma) from params
    estimated from scipy.stats.lognorm.fit(obs_data)'''
    shape, loc, scale = params
    mu, sigma = np.log(scale), shape
    return mu, sigma

# Dev

In [14]:
TLDf = os.path.join('example_data', 'obsTLD.txt')
TLD = pd.read_csv(TLDf, sep=', ', engine='python')
TLD['purp'] = TLD['Purpose'] + '_' + TLD['Direction']
TLD = TLD[['purp', 'Trips', 'StageDistance']]
TLD['StageDistance'] = TLD['StageDistance'].apply(nband(1))
TLD = TLD.groupby(['purp','StageDistance']).sum()['Trips'].unstack(1).T
# mid interval:
TLD.index = TLD.index.values + (pd.Series(TLD.index.values).shift(-1) - TLD.index.values) / 2
#TLD.loc[0,:] = 0 ##THIS SEEM TO PREVENT A GOOD FIT
TLD = TLD.sort_index()
TLD = truncate_TLD(TLD, 100)
TLD

purp,Business_FH,Business_NHB,Business_TH,Commuting_FH,Commuting_TH,Other_FH,Other_NHB,Other_TH
0.5,40.97080,34.91972,26.45936,257.60701,219.81880,2284.58024,840.99699,2139.71091
1.5,124.16246,167.82784,131.00452,1100.10610,970.23366,5634.56027,2196.08065,5707.37711
2.5,114.78300,183.78735,100.48967,1247.68330,1108.91831,4842.61562,1926.36506,4953.25923
3.5,112.16961,174.71845,95.53958,1120.70646,1041.01498,3812.47738,1544.85841,3794.42781
4.5,74.40131,115.35999,79.68665,742.13750,674.28828,2353.73826,982.82230,2445.05143
5.5,75.97420,133.61455,75.99749,630.39313,586.85777,2067.95749,878.90447,2083.79434
6.5,80.43625,94.93315,76.83523,529.32639,509.15259,1624.60643,683.57418,1704.66804
7.5,43.40328,54.09755,45.02179,585.15745,513.32134,1157.41313,556.09186,1292.66830
8.5,41.52530,56.95351,37.88665,481.97420,448.35972,1059.18340,503.47057,1178.80662
9.5,33.53091,40.86349,33.58387,329.85820,291.92097,667.17978,271.64842,658.33165


In [15]:
OutputName = os.path.join('example_outputs', 'TLD.png')
TLD_to_JPG(TLD, OutputName)

In [16]:
Y = TLD['Business_FH'].dropna()
Y = TLD['Business_FH'].fillna(0)
X = Y.index.values
Ynorm = Y.apply(lambda x: x/Y.sum())

In [25]:
#TODO: turn this into a function
tfuncs = 'lognorm gamma exponnorm exponweib exponpow pareto pearson3'.split()
tfuncs = 'gamma exponnorm exponpow'.split()
OutputName = os.path.join('example_outputs', 'TLD.png')

df_display = pd.DataFrame()
df_display['Business_FH_norm'] = Ynorm
for funcn in tfuncs:
    distrib = getattr(stats, funcn)
    params = distrib.fit(Y)
    df_display[funcn] = distrib.pdf(X, *params)

params = tanner.fit(X, Ynorm, p0=[-1,5])
df_display['tanner'] = tanner._pdf(X, *params)

TLD_to_JPG(df_display, OutputName)
df_display

,Business_FH_norm,gamma,exponnorm,exponpow,tanner
0.5,0.027769,0.066365,0.065519,0.142874,1.724509e-03
1.5,0.084155,0.046142,0.061222,0.065506,9.395682e-02
2.5,0.077798,0.038521,0.057207,0.045117,9.799350e-02
3.5,0.076027,0.033951,0.053455,0.035095,3.161279e-02
4.5,0.050428,0.030727,0.049949,0.028982,5.749466e-03
5.5,0.051494,0.028251,0.046673,0.024804,7.436521e-04
6.5,0.054518,0.026248,0.043612,0.021741,7.671085e-05
7.5,0.029418,0.024569,0.040752,0.019384,6.734925e-06
8.5,0.028145,0.023127,0.038080,0.017506,5.240590e-07
9.5,0.022727,0.021863,0.035582,0.015969,3.712707e-08
